## Reductions Redux

### Learning objectives

In this lab you will learn:

- Parallel reduction strategies
- Warp shuffle operations

After completing this lab, you will be able to understand how to write reduction operations to achieve maximum throughput.

### Prerequisites

It is assumed that participants have experience with:

- How to launch CUDA kernels that use both blocks and threads


- Basic memory management (`cudaMalloc()`, `cudaFree()`, `cudaMemcpy`)


- How to use shared memory for user-managed caching


- Selecting an optimal kernel launch configuration for exposing massive parallelism


- Achieving optimal global memory throughput (coalescing) and shared memory thoughput (avoiding bank conflicts)


- Atomic operations in CUDA such as `atomicAdd()`

## Classical Parallel Reduction

In a previous lab we saw that atomics are extremely convenient for reductions, but you should assume you cannot achieve peak memory bandwidth when using them (especially when using them simultaneously among many threads). So let's look at other strategies for reductions that minimize or even eliminate the need for atomics. Parallel reductions are extremely commonly used, so it's worth having some tools in our toolkit for addressing reduction-style problems in addition to transformation-style problems.

Returning to our example of calculating the sum of an array, how could we do this in a way that avoids the use of atomic operation but also uses as many threads as possible? A typical approach is to use a tree-based approach. The methodology is to perform pairwise reductions. In the example below, we have an array with eight items. We could start by summing up 3+1 to get 4, 7+0 to get 7, 4+1 to get 5, and 6+3 to get 9. We would then have reduced our dataset by a factor of 2. We can again do pairwise additions to reduce our intermediate dataset to size 2, and then a final pairwise addition. While the final step only uses a single thread, notice that the initial step could use a number of threads that is comparable in magnitude to the number of items in the dataset.

![](images/tree_reduction.png)

### Problem: Global Synchronization

So, we want to realize this algorithmically in CUDA. But before we discuss that, think about how the result of 11 was arrived at on the left side of the figure. It was produced by summing 4 and 7, which were each produced by adding 3 and 1, and 7 and 0, respectively. Let's focus on one thread, the one that added 3 and 1 to get 4, and say that we want this same thread to produce the output leading to 11. Now, in CUDA we've asserted that threads operate in no particular defined order, and that if you want thread ordering to occur, it must be imposed by you as the programmer, through the use of barrier operations such as synchronization. So what happens if the thread responsible for producing the result of 7 has not executed yet? Then we'll have a race condition and won't get the right answer. So we want to assert that any given row in the tree must be completely done before moving on to the next row.

Said another way, we want to synchronize all our threads between each set of additions. Now, if all of our threads resided within a single threadblock, we have a tool for this synchronization: `__syncthreads()`. But in general we want to solve problems that are larger than the maximum number of threads in a block, and we'll be required to use many threadblocks. So in order to realize this algorithm, we would need a way to synchronize *across* threadblocks, not just *within* threadblocks, to achieve a global synchronization of the grid. And we haven't yet discussed any ways to synchronize an entire grid.

There are some solutions to this problem though. A straightforward approach is that we could decompose each row or level in the tree into separate kernels. In the CUDA programming model, kernels that are launched in sequence serialize with respect to each other: if two kernels are launched in a row, the second kernel will not begin executing until the first one completes executing (there are exceptions to this, which will be discussed in a later module). Since the end of a kernel occurs when all threads have completed execution, this creates an effective global synchronization point. Note that the overhead involved with launching a kernel is small (on the order of microseconds) but it is not zero. This becomes a relevant cost to consider when launching small amounts of work, and the last few levels of our tree would be quite small in this approach.

Another solution is to perform tree reductions within threadblocks (where you can synchronize to avoid race conditions), and then use atomic operations on the result of each threadblock. (A more intricate method is a threadblock-draining approach, using atomic operations to determine which is the last threadblock to remain alive in the grid -- see the [threadFenceReduction](https://docs.nvidia.com/cuda/cuda-samples/index.html#threadfencereduction) CUDA sample code.) Finally, you can use the [cooperative groups](https://devblogs.nvidia.com/cooperative-groups/) programming model which allows you to create arbitrary decompositions of thread groups directly in CUDA. You could obtain a handle to the set of all threads on the grid, and then issue a synchronization on that group. (However, the kernel needs to be launched [using a particular API](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#grid-synchronization-cg).)

### Sequential Addressing

Since most of these approaches involve reductions within a threadblock, let's examine that in some detail. Our approach will be to horizontally decompose our tree across threadblocks, and just focus on how to perform the reduction of a part of the tree within a single threadblock. Here's a code snippet which does this, using shared memory (this code will be running simultaneously in many threadblocks, of course):

```
for (unsigned int s = blockDim.x / 2; s > 0; s /= 2) {
    if (threadIdx.x < s) {
        sdata[threadIdx.x] += sdata[threadIdx.x + s];
    }
    __syncthreads(); // Note: outside the conditional, so all threads must wait.
}
```

This code will perform several iterations, with each iteration corresponding to moving vertically doward in the tree. The for loop starts with a counter `s` equal to half the number of threads in the block. For every thread with index less than `s`, that thread takes a value from shared memory and adds it to another value in shared memory. Both locations accessed are indexed using the same thread ID, with one offset relative to the other. For example, let's say the size of the block is 64: thread 0 takes the value in location 32 and adds it to the value at location 0. Thread 1 adds index 33 to index 1, etc.  This performs a "sweep" operation, which is depicted below.

![](images/sequential_addressing.png)

In this diagram, each location is a value in shared memory, and in each sweep, we are effectively condensing the data down, moving the data from the right side to the left side. The sweep proceeds from top to bottom, with each row corresponding to an iteration in the for loop. However before we can proceed from one level sweep to the next, we must synchronize all threads in the block, using `__syncthreads()`. In each successive sweep, `s` has been divided by 2, so the amount of work to do is halving each time. At the end of the for loop, the final result is a partial sum, the sum of values in that threadblock, and it is located in `s[0]`.

This is not the only threading strategy we could use; for example, we could have had each thread add the value immediately adjacent to it. However, most threading strategies other than the sequential addressing approach we have chosen here will encounter bank conflicts in shared memory, and will probably achieve suboptimal performance.

### Detour: Grid-Stride Loops

We're trying to assemble a complete kernel to perform this sum reduction, so we need more tools than just the threadblock level code we just showed. Another useful tool is a **grid-stride loop**. In this approach, the size of the problem we're working on is decoupled from the size of the grid we use, and in addition to being able to solve problems where the number of threads is the same size as the input data, we can solve problems where the input data is smaller or larger than the number of threads. This allows you to write flexible, reusable code. (In some cases it can also be more efficient than the strategy of launching as many threads as there are items in the dataset, although this benefit is usually only at the few percent level.)

So we have a grid of threads with a particular width, and at any one time will process a set of data equal to the width of that grid. If there's more data to process, then we're going to have all the threads in the grid stride to the next set of locations in memory, and repeat their work. The size of the stride is equal to the total number of threads in the grid. So if the number of threads in the grid is `M`, then thread 0 will work on element 0, M-1, 2M-1, 3M-1, etc.

![](images/grid_stride_loop.png)

This code sample demonstrates an example of a grid-stride loop: using a while loop, we initialize our `idx` variable to the globally unique thread in the grid, and then continue looping as long as `idx < N`. In each iteration of the while loop, we add a stride equal to the size of the grid (`blockDim.x * gridDim.x`). Any thread will continue looping until its index counter is larger than the number of items in the dataset, at which point it will be done. (This also neatly handles the case where `N` is not an integer multiple of the number of threads in the grid.)

The work done in this loop is to add data from the global memory array (`gdata`) to a shared memory array (`sdata`). In this way, each thread in the grid computes a running sum of all of the items it gets assigned to in this strided loop. Once we have computed this initial processing step, then we can begin to perform our reductions within threadblocks. Notice that if the number of threads in the grid is equal to the number of items in the dataset, this loop executes exactly once for all threads, and has each thread load one item from global memory into shared memory, which is exactly what you expect. It's useful to do these sorts of sanity checks; convince yourself that this code also works in other special cases, like when the total number of threads in the grid is 1, or when it is equal to the number of threads in a block (so you're just launching one block).

### Putting It All Together

We now have all the tools to look at a complete kernel for a tree reduction. We allocate shared memory and initialize it to zero. We obtain our globally unique thread index as a starting point, and then do a grid-stride loop through the data, computing a running sum for each thread. We then start the parallel sweep operation. Note that we have reversed the order of `__syncthreads()` relative to the previous code sample. We have to do this because we already have done some shared memory writes in the grid-stride loop, and we need to synchronize the threadblock an initial time before reading the data.

At the end, we have the result that the sum for each threadblock is stored in `s[0]`, the first location in our shared memory array. Then we have the thread with `threadIdx.x == 0` write that result to an array in global memory (`out`), to a unique index for the block it is in. This implies that the size of `out` must be at least as large as the number of threadblocks in the grid.

```
__global__ void reduce(float* gdata, float* out) 
{
    __shared__ float sdata[BLOCK_SIZE];
    int tid = threadIdx.x;
    sdata[tid] = 0.0f;
    
    while (idx < N) { // grid-stride loop to load data
        sdata[tid] += gdata[idx];
        idx += gridDim.x * blockDim.x;
    }
    
    for (unsigned int s = blockDim.x / 2; s > 0; s /= 2) {
        __syncthreads();
        if (tid < s) { // parallel sweep reduction
            sdata[tid] += sdata[tid + s];
        }
    }
    
    if (tid == 0) out[blockIdx.x] = sdata[0];
}
```

So what we end up with is an array of partial sums in global memory. One approach would then be to launch a second kernel, performing another tree reduction on these partial sums. As long as the number of partial sums was less than the number of threads in a block, we could just launch the kernel with a single block and it would be sufficient for computing the final answer. Conveniently, we could use the exact same code to do it -- because we used a grid-stride loop to load the data into shared memory, our code will work correctly even if we use only a single block.

But instead of launching a second kernel let's look at the strategy of using an atomic operation at the end.

```
__global__ void reduce(float* gdata, float* out) 
{ 
    __shared__ float sdata[BLOCK_SIZE];
    int tid = threadIdx.x;
    sdata[tid] = 0.0f;
    
    while (idx < N) { // grid-stride loop to load data
        sdata[tid] += gdata[idx];
        idx += gridDim.x * blockDim.x;
    }
    
    for (unsigned int s = blockDim.x / 2; s > 0; s /= 2) {
        __syncthreads();
        if (tid < s) { // parallel sweep reduction
            sdata[tid] += sdata[tid + s];
        }
    }
    
    if (tid == 0) atomicAdd(out, sdata[0]);
}
```

Now every block writes to a *single* location, `out`, and the output after the atomic operation has been applied by all blocks will be the final sum of the entire array. No postprocessing is necessary. And compared to the approach of using a fully atomic reduction, this method substantially reduces the pressure on the atomic hardware on the GPU. The threadblocks do most of the work, efficiently summing up data from the dataset by striding through it and then performing an intra-block reduction. This could reduce the number of simultaneous atomic operations from millions to thousands, perhaps, and so we will achieve substantially higher throughput during the atomic operations. This approach to writing a reduction kernel is likely one of the best ways to do it.

Atomics aren't the answer to everything. They aren't supported for all possible datatypes (and historically the support for various datatypes like double precision has changed over time), and they also don't support all possible reduction operations (one could imagine a sufficiently complicated user-defined reduction that cannot be decomposed into the fundamental atomic operations like addition, subtraction, max/min, etc.). Nevertheless they are a strong tool.

## Exercise

For this exercise, you are given a fully functional sum reduction code in [exercises/max_reduction.cu](exercises/max_reduction.cu), similar to the code used above, except that we will use the two-stage reduction method without an atomic finish. If you wish you can compile and run it as-is to see how it works. Your task is to modify the kernel so that it creates a proper max-finding reduction. That means that the kernel should report the maximum value in the data set, rather than the sum of the data set. You are expected to use a similar parallel-sweep-reduction technique. If you need help, refer to the solution, [solutions/max_reduction.cu](solutions/max_reduction.cu).

In [ ]:
!nvcc -arch=native -o max_reduction exercises/max_reduction.cu; ./max_reduction

##  Warp Shuffle

We'll close this module with warp shuffles.

### Inter-Thread Communication: So Far

In the reduction examples we've shown so far, we've been using shared memory to communicate between one threads. This requires two operations: one thread has to write to shared memory, and another has to read from it (two instructions total). Additionally, we may require a synchronization between these two operations; we certainly do in the tree reduction algorithm we presented.

![](images/inter_thread_communication_1.png)

Wouldn't it be nice if, instead of this pattern, we could simply communicate from one thread to another, like this?

![](images/inter_thread_communication_2.png)

### Introducing Warp Shuffle

Warp shuffle operations allow for direct communication between threads within a warp. Until now, we've talked about how threads operate in groups called warps, but noted that the concept of a warp wasn't part of our CUDA programming model. Warp shuffle operations, however, directly inject the concept of a warp into the programming model, so you have to know what warps are (groups of 32 threads that are subsets of a threadblock that operate essentially in lockstep). A warp shuffle operation allows you to communicate from one thread to another using a single instruction in a single cycle, *without* using shared memory.

There are various supported movement patterns. Some examples are:

- `__shfl_sync()`: copy from lane ID (arbitrary pattern)

- `__shfl_xor_sync()`: copy from calculated line ID (calculated pattern)

- `__shfl_up_sync()`: copy from delta/offset lower lane

- `__shfl_down_sync()`: copy from delta/offset higher lane

![](images/warp_shuffle.png)

These instructions require source and destination threads to both participate in the instruction, and all take as argument a "mask" that determines which threads participate in the shuffle operation. It's OK to use these instructions within conditional code that not all threads in a warp participate in, but the mask has to be consistent, and the source and destination should both be participating.

### Warp Shuffle Reduction

Now we can write a fully worked out example of a single reduction kernel that uses warp shuffles. As before, we use a grid-stride loop to load data, but instead of loading to shared memory we load to a thread-local variable (`val`). We then define some useful values: `mask` will determine which threads participate in our warp shuffle instructions, and we're setting it such that all threads participate; `lane` determines which thread we are within a warp (so every thread will have a lane between 0 and 31); and, `warpID` determines which warp in the block our thread is in.

The for loop for our warp shuffle reduction looks similar in structure to the for loop we used before (starting at `warpSize / 2` and dividing by 2 each time), but now we're adding data from `val` in one thread to `val` in another thread (again, occurring fully in thread-local memory -- in registers -- rather than requiring other memory like shared memory). Conceptually this is very similar to the sweep operation we previously saw: in the first pass, thread 16 adds its result to thread 0, thread 17 adds its result to thread 1, etc.; in the second pass, thread 8 adds its result to thread 0, thread 9 adds its result to thread 1, etc.; and so on until thread 0 finally has the final sum for the warp.

```
__global__ void reduce_ws(float* gdata, float* out) 
{
    __shared__ float sdata[32];
    
    int tid = threadIdx.x;
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    
    float val = 0.0f;
    unsigned mask = 0xFFFFFFFFU;
    int lane = threadIdx.x % warpSize;
    int warpID = threadIdx.x / warpSize;
    
    while (idx < N) {  // Grid-stride loop to load data
        val += gdata[idx];
        idx += gridDim.x * blockDim.x;  
    }
    
    // First warp-shuffle reduction
    for (int offset = warpSize / 2; offset > 0; offset /= 2) {
        val += __shfl_down_sync(mask, val, offset);
    }
        
    if (lane == 0) sdata[warpID] = val; // Put warp results in shared memory
    
    __syncthreads();
    
    // Hereafter, just warp 0
    if (warpID == 0) {
        // Reload val from shared mem if warp existed
        val = (tid < blockDim.x / warpSize) ? sdata[lane] : 0;
        
        // Final warp-shuffle reduction
        for (int offset = warpSize / 2; offset > 0; offset /= 2) {
            val += __shfl_down_sync(mask, val, offset);
        }
        
        if (tid == 0) atomicAdd(out, val);
    }
}
```

If we are the first thread in a warp, we write our partial sum into shared memory, into a unique location for each warp. Since we know the maximum number of threads per block is 1024, and there are 32 threads per warp, we need at most 32 locations in shared memory to do this (hence the hardcoded requirement at the beginning of the kernel). Then we synchronize the block to ensure all warps have had the chance to write their partial sums before we proceed.

We can complete the kernel with a final block of code that only needs to execute with a single warp (we arbitrarily choose warp 0), since the maximum number of elements in `sdata` is 32 and we have 32 threads in a warp. This warp will perform a final tree reduction on our partial warp sums, returning the partial sum for the block. Finally, if we are the first thread in the block (which will be in the first warp in the block), we perform an atomic operation to write our threadblock's result to global memory.

### Warp Shuffle Benefits

Some benefits of using warp shuffles include:

- We can reduce (or in some implementations eliminate) the amount of shared memory used by the kernel
  - Shared memory can be a limiter to occupancy, so this can improve performance


- We have replaced two or more instructions with a single instruction


- We have reduced the need for explicit synchronization compared to the shared memory sweep (synchronizations can be expensive)

### Warp Shuffle Tips and Tricks

What else can we do with this? As an example, we can broadcast a value from one thread in a warp to all other threads in a warp in a single instruction. We can also perform certain common algorithms like prefix sums or sorting efficiently within a warp. Finally, in cases where all threads in a warp might be performing atomic operations on the same location in memory, it can be more efficient to first reduce among those values within a warp, and then elect only a single thread within a warp to perform the atomic operation, as this reduces pressure on the atomic subsystem. In a sense, this is exactly what we did with the tree reduction, compared to the naive method of just using a fully atomic implementation.

## Exercise

The code [exercises/reductions.cu](exercises/reductions.cu) has fully worked out implementations of three parallel sum reduction implementations we previously discussed: a fully atomic reduction, a classical tree reduction using the sweep method (with an atomic finish rather than a second kernel), and a tree reduction using warp shuffle operations (still finished with atomics).

Let's compile and run it, making sure the code gets the correct answer (as verified by its stdout messages).

In [ ]:
!nvcc -arch=native -o reductions exercises/reductions.cu; ./reductions

Now let's run the code with Nsight Compute in its default profiling mode, which is sufficient. We want to compare kernel execution times. What do you notice about kernel execution times? Probably, you won't see much difference between the parallel reduction with atomics and the warp shuffle with atomics kernel. Can you theorize why this may be?

In [ ]:
!ncu ./reductions

Our objective with these will be to approach theoretical limits. The theoretical limit for a typical reduction would be determined by the memory bandwidth of the GPU. To calculate the attained memory bandwidth of this kernel, divide the total data size in bytes (use N from the code in your calculation) by the execution time (which you can get from the profiler). (Or, if you like, look at the `SOL DRAM` output from the profiler.) How does this number compare to the memory bandwidth of the GPU you are running on? (If you don't know the GPU's peak bandwidth, the CUDA [bandwidthTest sample code](https://docs.nvidia.com/cuda/cuda-samples/index.html#bandwidth-test) will give you an estimate.)

Now edit the code to change N from 8 million to 163840 (== 640 * 256).

Recompile and re-run the code with profiling. Is there a bigger percentage difference between the execution time of the reduce_a and reduce_ws kernel? Why might this be?

As a bonus exercise, edit the code to change N from 8 million to 32 million. recompile and run. What happened? Why?

## Exercise: Revisiting Matrix Row and Column Sums

For this exercise, we start with the [matrix_sums.cu](exercises/matrix_sums.cu) code from the previous module. As you may recall, the row_sums kernel was reading the same data set as the column_sums kernel, but running noticeably slower. We now have some ideas how to fix it. See if you can implement a reduction-per-row, to allow the row-sum kernel to approach the performance of the column sum kernel. There are probably several ways to tackle this problem. To see one approach, refer to the solution, [solutions/matrix_sums.cu](solutions/matrix_sums.cu).

In [ ]:
!nvcc -arch=native -o matrix_sums exercises/matrix_sums.cu; ./matrix_sums

In [ ]:
!ncu ./matrix_sums

## Summary

In this lab you have learned:

- How to implement classic parallel reductions using atomic operations and sweep-style methods

Now that you can implement highly performant transformations and reductions, you're well equipped to tackle a wide range of problems in CUDA that you might encounter.

## Further Study

[Parallel reduction](https://developer.download.nvidia.com/assets/cuda/files/reduction.pdf)

[Warp-shuffle and reduction](https://devblogs.nvidia.com/faster-parallel-reductions-kepler/)

[CUDA Cooperative Groups](https://devblogs.nvidia.com/cooperative-groups/)

[Grid-stride loops](https://devblogs.nvidia.com/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/)

[Floating point](https://developer.nvidia.com/sites/default/files/akamai/cuda/files/NVIDIA-CUDA-Floating-Point.pdf)

[CUDA Sample Codes](https://docs.nvidia.com/cuda/cuda-samples/index.html): Reduction, threadFenceReduction, reductionMultiBlockCG

## Lab Materials

You can download this notebook using the `File > Download as > Notebook (.ipnyb)` menu item. Source code files can be downloaded from the `File > Download` menu item after opening them.